# import the data

$$
f=x^2 + \sqrt {y} + z
$$

In [216]:
import pandas as pd

In [217]:
df = pd.read_csv("data/result_testdata.csv", delimiter=",")
df

,x,y,z,f
0,0.876300,0.087160,0.678294,1.741426
1,0.506543,0.427771,0.117943,1.028571
2,0.469181,0.208947,0.862431,1.539668
3,0.997632,0.038999,0.628393,1.821145
4,0.680144,0.432733,0.571025,1.691445
...,...,...,...,...
569,0.213704,0.183989,0.577722,1.052330
570,0.727660,0.021602,0.820556,1.497021
571,0.452470,0.848223,0.896740,2.022459
572,0.470553,0.443268,0.204643,1.091847


# build the network

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

In [218]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import autograd
import math
from sklearn.metrics import mean_squared_error

In [219]:
X = df.iloc[:,:3]
y = df.iloc[:,3]
# y = df.iloc[:,:3].sum(axis=1) 

In [220]:
X = torch.tensor(X.values.astype(np.float32), requires_grad=True)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [221]:
X.shape

torch.Size([574, 3])

In [222]:
y.shape

torch.Size([574, 1])

In [223]:
class EstimateF(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden11 = nn.Linear(3, 160)
        self.act11 = nn.ReLU()
        self.hidden12 = nn.Linear(160, 80)
        self.act12 = nn.ReLU()
        self.output1 = nn.Linear(80, 16)
        self.act_output1 =  nn.Linear(16, 1)
        self.hidden21 = nn.Linear(3, 160)
        self.act21 = nn.ReLU()
        self.hidden22 = nn.Linear(160, 80)
        self.act22 = nn.ReLU()
        self.output2 = nn.Linear(80, 16)
        self.act_output2 =  nn.Linear(16, 3)
    def forward(self, x):
        x1 = self.act11(self.hidden11(x))
        x1 = self.act12(self.hidden12(x1))
        x1 = self.act_output1(self.output1(x1))
        x2 = self.act21(self.hidden21(x))
        x2 = self.act22(self.hidden22(x2))
        x2 = self.act_output2(self.output2(x2))
        return x1,x2
 
model = EstimateF()
print(model)

EstimateF(
  (hidden11): Linear(in_features=3, out_features=160, bias=True)
  (act11): ReLU()
  (hidden12): Linear(in_features=160, out_features=80, bias=True)
  (act12): ReLU()
  (output1): Linear(in_features=80, out_features=16, bias=True)
  (act_output1): Linear(in_features=16, out_features=1, bias=True)
  (hidden21): Linear(in_features=3, out_features=160, bias=True)
  (act21): ReLU()
  (hidden22): Linear(in_features=160, out_features=80, bias=True)
  (act22): ReLU()
  (output2): Linear(in_features=80, out_features=16, bias=True)
  (act_output2): Linear(in_features=16, out_features=3, bias=True)
)


In [224]:
loss_fn = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 50000
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size].clone()
        # Xbatch.requires_grad=True
        y_pred1, y_pred2 = model(Xbatch)
        ybatch = y[i:i+batch_size]
        ybatch2 = autograd.grad(y_pred1,Xbatch,torch.ones([Xbatch.shape[0], 1]), retain_graph=True, create_graph=True)[0] #first derivative
        loss = loss_fn(y_pred1, ybatch) + loss_fn(torch.pow(Xbatch,y_pred2-torch.ones(y_pred2.size())).mul(y_pred2), ybatch2)
        # if epoch==0 and i==0:
        #     # print(y_pred1)
        #     print(Xbatch)
        #     print(y_pred2)
        #     # print(loss_fn(y_pred1, ybatch))
        #     print(y_pred2-torch.ones(y_pred2.size()))
        #     print(torch.pow(Xbatch,y_pred2-torch.ones(y_pred2.size())))
        #     print(torch.pow(Xbatch,y_pred2-torch.ones(y_pred2.size())).mul(y_pred2))                   
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

tensor([[0.8763, 0.0872, 0.6783],
        [0.5065, 0.4278, 0.1179],
        [0.4692, 0.2089, 0.8624],
        [0.9976, 0.0390, 0.6284],
        [0.6801, 0.4327, 0.5710],
        [0.5288, 0.2589, 0.6544],
        [0.7432, 0.7144, 0.2645],
        [0.3437, 0.6037, 0.3529],
        [0.1806, 0.1608, 0.3050],
        [0.9655, 0.0042, 0.8323]], grad_fn=<CloneBackward0>)
tensor([[ 0.0393, -0.2009, -0.0276],
        [-0.0132, -0.1992, -0.0237],
        [ 0.0420, -0.2026, -0.0278],
        [ 0.0412, -0.1942, -0.0231],
        [ 0.0171, -0.2089, -0.0394],
        [ 0.0255, -0.2033, -0.0308],
        [-0.0072, -0.2077, -0.0336],
        [ 0.0023, -0.2038, -0.0286],
        [ 0.0061, -0.1866, -0.0169],
        [ 0.0537, -0.2032, -0.0294]], grad_fn=<AddmmBackward0>)
tensor([[-0.9607, -1.2009, -1.0276],
        [-1.0132, -1.1992, -1.0237],
        [-0.9580, -1.2026, -1.0278],
        [-0.9588, -1.1942, -1.0231],
        [-0.9829, -1.2089, -1.0394],
        [-0.9745, -1.2033, -1.0308],
        [-1.00

In [225]:
# compute accuracy (no_grad is optional)
# with torch.no_grad():
#     y_pred = model(X)

X.requires_grad_()
X.retain_grad()
y_pred1,y_pred2 = model(X)

trainScore = math.sqrt(mean_squared_error(y_pred1.detach().numpy(),y.detach().numpy()))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 0.07 RMSE


# test the auto gradient

$$
f=x^2 + \sqrt {y} + z
$$

$$
f_x'=2x
$$

$$
f_y'=\frac{1}{2} * y ^ {-\frac{1}{2}}
$$

$$
f_z'=1
$$

In [226]:
X[0]

tensor([0.8763, 0.0872, 0.6783], grad_fn=<SelectBackward0>)

In [227]:
y[0]

tensor([1.7414])

In [228]:
y1,y2=model(X[0])
y1

tensor([1.7508], grad_fn=<AddBackward0>)

In [229]:
y2

tensor([1.2803, 0.0771, 1.0055], grad_fn=<AddBackward0>)

In [230]:
y_pred1,y_pred2=model(X)


In [231]:
autograd.grad(model(X)[0],X,torch.ones([X.shape[0], 1]), retain_graph=True, create_graph=True) #first derivative

(tensor([[1.2055, 0.7925, 0.9545],
         [1.0223, 0.7736, 0.9706],
         [0.9089, 0.7737, 0.9625],
         ...,
         [0.9466, 0.6641, 0.9641],
         [0.9905, 0.7622, 0.9715],
         [0.9084, 0.7725, 0.9677]], grad_fn=<TBackward0>),)

good match!